# Ball Tracking Dataset 

In [ ]:
import os
import shutil
import glob
import csv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [ ]:
def copy_and_rename_images(src_dir, dest_dir, file_extension=".jpg"):
    """
    Copies and renames images from the source directory to the destination directory
    by removing leading zeros from filenames.

    Parameters:
    - src_dir (str): The source directory containing the images to copy.
    - dest_dir (str): The destination directory where the images will be copied and renamed.
    - file_extension (str): The file extension of the images to copy (default is ".png").
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for filename in os.listdir(src_dir):
        if filename.endswith(file_extension):
            # Remove the leading zeros from the numeric part of the filename
            base_name = filename[:-len(file_extension)]  # Get the part without the extension
            new_base_name = str(int(base_name))  # Convert to int and back to str to remove leading zeros
            new_filename = new_base_name + file_extension  # Add the file extension back
            
            src_file = os.path.join(src_dir, filename)
            dest_file = os.path.join(dest_dir, new_filename)
            shutil.copy(src_file, dest_file)
#             print(f"Copied and renamed {src_file} to {dest_file}")


In [ ]:
import re

# Define a function to read the .ini file and extract the ball's trackletID
def get_ball_tracklet_id(file_path):
    ball_tracklet_id = None
    
    with open(file_path, 'r') as file:
        for line in file:
            if 'ball;' in line:
                match = re.search(r'trackletID_(\d+)= ball;(\d+)', line)
                if match:
                    ball_tracklet_id = match.group(1)
                    break
    
    return ball_tracklet_id

In [ ]:
def find_diagonal_intersection(top, left, height, width):
    intersection_x = left + width / 2
    intersection_y = top + height / 2
    return float(intersection_x), float(intersection_y)

In [ ]:
# Ensure output directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Preprocess single training example
def preprocess_single_training_example(dir_name ,match, output_file_name , dataset_type):
    pics_path = "/kaggle/input/football-tracking-data/{}/{}/{}/img1"
#     output_gt_path = "/kaggle/working/ball_tracking/test/{}/ground_truth"
    output_pics_path = "/kaggle/working/ball_tracking/{}/{}/frame/{}"
    label_path = "/kaggle/input/football-tracking-data/{}/{}/{}/gt/gt.txt"
    tracklet_id_path = "/kaggle/input/football-tracking-data/{}/{}/{}/gameinfo.ini"
    copy_and_rename_images(src_dir=pics_path.format(dataset_type,dataset_type,dir_name), dest_dir=output_pics_path.format(dataset_type,match ,dir_name) )
    # input_folder = os.path.join('D:/player_tracking/path/to/SoccerNet/tracking/test/test', str(dir_name) +"/"+"img1")
    input_folder = pics_path.format(dataset_type,dataset_type,dir_name)

    # txt_file =  os.path.join('D:/player_tracking/path/to/SoccerNet/tracking/test/test' , dir_name +"/" + '/gt/gt.txt')
    txt_file = label_path.format(dataset_type,dataset_type,dir_name)
    # Read the comma-separated text file into a DataFrame
    df = pd.read_csv(txt_file, header=None , names=["Frame" , "Track_id" , "left" , "top", "height" , "width" , "a" ,"b" , "c" , "d"])

    # file_path = os.path.join('D:/player_tracking/path/to/SoccerNet/tracking/test/test' , dir_name +"/" + 'gameinfo.ini')
    file_path = tracklet_id_path.format(dataset_type,dataset_type,dir_name)
    ball_tracklet_id = get_ball_tracklet_id(file_path)
    print(ball_tracklet_id)
    filtered_df = df[df['Track_id'] == int(ball_tracklet_id)]

    output_csv = pd.DataFrame()
    output_csv["Frame"] = [i for i in range(1,751)]

    output_csv['Visibility'] = output_csv['Frame'].apply(lambda x: 1 if x in filtered_df['Frame'].values else 0)
    output_csv["top"] = output_csv['Frame'].map(lambda x: filtered_df.loc[filtered_df['Frame'] == x, 'top'].values[0] if x in filtered_df['Frame'].values else 0)
    output_csv['left'] = output_csv['Frame'].map(lambda x: filtered_df.loc[filtered_df['Frame'] == x, 'left'].values[0] if x in filtered_df['Frame'].values else 0)
    output_csv["height"] = output_csv['Frame'].map(lambda x: filtered_df.loc[filtered_df['Frame'] == x, 'height'].values[0] if x in filtered_df['Frame'].values else 0)
    output_csv["width"] = output_csv['Frame'].map(lambda x: filtered_df.loc[filtered_df['Frame'] == x, 'width'].values[0] if x in filtered_df['Frame'].values else 0)
    output_csv[['X', 'Y']] = output_csv.apply(lambda row: pd.Series(find_diagonal_intersection(row['top'], row['left'], row['height'], row['width'])), axis=1)

    
    output_csv.drop(columns=['top', 'left', 'width', 'height'], inplace=True)
    print(output_csv.head())
    output_file_name = os.path.join(output_file_name,"csv")
    ensure_dir(output_file_name)
    output_csv_file = output_file_name+"/"+f"{dir_name}_ball.csv"

    output_csv.to_csv(output_csv_file ,index=False)
    return output_csv_file

In [ ]:
!pip install natsort

In [ ]:
train_dir_names = os.listdir("/kaggle/input/football-tracking-data/train/train")
test_dir_names = os.listdir("/kaggle/input/football-tracking-data/test/test")

In [ ]:
import natsort
train_dir_names = natsort.natsorted(train_dir_names)
test_dir_names = natsort.natsorted(test_dir_names)
print(len(test_dir_names) , len(train_dir_names))

### Train Dataset

In [ ]:
match_num =1
for i in train_dir_names:
    dataset_type = "train"
    match_num = match_num
    match="match"+ str(match_num)
    output_file_name = "/kaggle/working/ball_tracking/{}/{}".format(dataset_type,match)
    dir_name = i
    
    label_csv = preprocess_single_training_example(dir_name, match=match,  output_file_name=output_file_name , dataset_type=dataset_type)
    print(dir_name)
    print(match_num)
    match_num +=1

# Test dataset

In [ ]:
match_num =1
for i in test_dir_names:
    dataset_type = "test"
    match_num = match_num
    match="match"+ str(match_num)
    output_file_name = "/kaggle/working/ball_tracking/{}/{}".format(dataset_type,match)
    dir_name = i
    
    label_csv = preprocess_single_training_example(dir_name, match=match,  output_file_name=output_file_name , dataset_type=dataset_type)
    print(dir_name)
    print(match_num)
    match_num +=1